In [1]:
import api_nyt as nyt
import api_wikipedia as wiki
import manage_articles as mng
import statistics
import datetime
import pickle
import timeseries
import TextPreprocessing as txt
from datetime import date
import graphics
import matplotlib.pyplot as plt
import matching
import time
import word
import random
import numpy

articles = mng.load_articles("nyt2019.json")

In [4]:
liste = mng.get_articles_as_list(articles)
liste[0]

{'headline': '1919: The Year of the Crack-Up',
 'pub_date': '2019-01-01T00:00:03+0000',
 'web_url': 'https://www.nytimes.com/2018/12/31/opinion/1919-america.html',
 'abstract': 'From the Treaty of Versailles to Prohibition, the events of that year shaped America, and the world, for a century to come. ',
 'lead_paragraph': 'Throughout 2019, The New York Times Opinion section will publish an occasional series of essays on the ways in which the events of 1919 shaped the following century. ',
 'keywords': [{'name': 'subject',
   'value': 'United States Politics and Government',
   'rank': 1,
   'major': 'N'},
  {'name': 'subject', 'value': 'New Year', 'rank': 2, 'major': 'N'},
  {'name': 'organizations',
   'value': 'League of Nations',
   'rank': 3,
   'major': 'N'},
  {'name': 'persons', 'value': 'Wilson, Woodrow', 'rank': 4, 'major': 'N'},
  {'name': 'glocations', 'value': 'Europe', 'rank': 5, 'major': 'N'}],
 'document_type': 'article',
 'type_of_material': 'Op-Ed',
 'word_count': 2248

In [3]:
articles_shuffled = mng.shuffle_publicationdates(articles)
liste_shuffled = mng.get_articles_as_list(articles_shuffled)
liste_shuffled[0]

{'headline': '1919: The Year of the Crack-Up',
 'pub_date': '2019-08-05',
 'web_url': 'https://www.nytimes.com/2018/12/31/opinion/1919-america.html',
 'abstract': 'From the Treaty of Versailles to Prohibition, the events of that year shaped America, and the world, for a century to come. ',
 'lead_paragraph': 'Throughout 2019, The New York Times Opinion section will publish an occasional series of essays on the ways in which the events of 1919 shaped the following century. ',
 'keywords': [{'name': 'subject',
   'value': 'United States Politics and Government',
   'rank': 1,
   'major': 'N'},
  {'name': 'subject', 'value': 'New Year', 'rank': 2, 'major': 'N'},
  {'name': 'organizations',
   'value': 'League of Nations',
   'rank': 3,
   'major': 'N'},
  {'name': 'persons', 'value': 'Wilson, Woodrow', 'rank': 4, 'major': 'N'},
  {'name': 'glocations', 'value': 'Europe', 'rank': 5, 'major': 'N'}],
 'document_type': 'article',
 'type_of_material': 'Op-Ed',
 'word_count': 2248,
 'section_na

In [3]:
#headlines = mng.getWordCounts(articles,'headline')
#with open("wordCount2019.txt", "wb") as fp:
#    pickle.dump(headlines, fp)
#top = mng.getTopWordsForWeek(headlines)
#distinctWords = mng.getDistinctWords(headlines)

#headlines = []
#with open("wordCount2019.txt", "rb") as fp:
#    headlines = pickle.load(fp)
#distinctWords = mng.getDistinctWords(headlines)

articles_shuffled = mng.shuffle_publicationdates(articles)
wordCounts = mng.getWordCounts(articles)
distinctWords = mng.getDistinctWords(wordCounts)
wordCounts_shuffled = mng.getWordCounts(articles_shuffled)
distinctWords_shuffled = mng.getDistinctWords(wordCounts_shuffled)

countsPerWeek = []
for w in distinctWords:
    countsPerWeek.append((w,mng.getCountPerWeek(wordCounts,w)))

countsPerWeek_shuffled = []
for w in distinctWords_shuffled:
    countsPerWeek_shuffled.append((w,mng.getCountPerWeek(wordCounts_shuffled,w)))    

words = []
for c in countsPerWeek:
    words.append(word.Word(c[0],ts_articles=timeseries.Timeseries(c[1])))
words = sorted(words, key=lambda x: sum(x.ts_articles.getCounts()), reverse=True)

words_shuffled = []
for c in countsPerWeek_shuffled:
    words_shuffled.append(word.Word(c[0],ts_articles=timeseries.Timeseries(c[1])))
words_shuffled = sorted(words_shuffled, key=lambda x: sum(x.ts_articles.getCounts()), reverse=True)

In [12]:
words[0].ts_articles.getDates()

[datetime.date(2019, 8, 19),
 datetime.date(2019, 2, 4),
 datetime.date(2019, 5, 6),
 datetime.date(2019, 12, 23),
 datetime.date(2019, 2, 18),
 datetime.date(2019, 7, 22),
 datetime.date(2019, 4, 22),
 datetime.date(2019, 4, 8),
 datetime.date(2019, 6, 17),
 datetime.date(2019, 9, 2),
 datetime.date(2019, 10, 28),
 datetime.date(2019, 5, 13),
 datetime.date(2019, 11, 11),
 datetime.date(2019, 2, 25),
 datetime.date(2019, 7, 8),
 datetime.date(2019, 3, 4),
 datetime.date(2019, 10, 7),
 datetime.date(2019, 1, 21),
 datetime.date(2019, 1, 28),
 datetime.date(2018, 12, 31),
 datetime.date(2019, 10, 14),
 datetime.date(2019, 1, 7),
 datetime.date(2019, 9, 23),
 datetime.date(2019, 12, 16),
 datetime.date(2019, 11, 4),
 datetime.date(2019, 8, 5),
 datetime.date(2019, 12, 2),
 datetime.date(2019, 6, 3),
 datetime.date(2019, 3, 25),
 datetime.date(2019, 5, 20),
 datetime.date(2019, 12, 9),
 datetime.date(2019, 3, 18),
 datetime.date(2019, 11, 18),
 datetime.date(2019, 5, 27),
 datetime.date(2

In [3]:
top_amount = 150

keywords = []
for i in range(top_amount):
    keywords.append(words[i].keyword)

m = matching.groupmatch(keywords, articles)

for key in m.keys():
    #Following line from 
    #https://stackoverflow.com/questions/7125467/find-object-in-list-that-has-attribute-equal-to-some-value-that-meets-any-condi
    word = next((x for x in words if x.keyword == key), None)
    query = m[key]['query']
    page = m[key]['link'][1]
    
    wiki_counts = wiki.get_counts(page, word.ts_articles.getStartDate(), word.ts_articles.getEndDate(),"en")
    if wiki_counts is not None:
        word.coocKeywords = query
        word.wikipediaSite = page
        word.ts_wiki = timeseries.parseWikipediaCounts(wiki_counts)
    


404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found


In [4]:
words_analyze = [x for x in words if x.wikipediaSite != ""]

corr = []
corr_rand = []

a = [x for x in range(len(words_analyze))] 
b = [x for x in range(len(words_analyze))] 
random.shuffle(b)

#remove words
for w in words_analyze:
    ts_a, ts_w = timeseries.alignTimeseries(w.ts_articles,w.ts_wiki)
    if len(ts_w.getCounts())<2 or len(ts_a.getCounts())<2:
        words_analyze.remove(w)
    else:
        corr.append(statistics.getCorrelation(ts_a.getCounts(),ts_w.getCounts()))


In [5]:
import numpy
print(numpy.mean(corr))

0.24973154623852784
